In [1]:
###### necessary packages
import torch
import torchvision
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from autogluon.tabular import TabularDataset, TabularPredictor

In [2]:
#ordering predicting vectors 
def find_stable_point():
    
    res = np.zeros(625)-99
    b=[]
    xm=[]
    ym=[]
    filename= './sr_C1_20/800rpm_100kPa_30C_Swirl=20_B00006_PIV_MP(2x32x32_50ov)=unknown/model-3-3-prediction-3-to-6/x/1-raw.dat'
    data=pd.read_csv(filename,index_col = 0).values
    for k in range(25):
        for i in range(1,24):
            for j in range(1,24):
                #for any sequence in i and j
                if data[i][j]!=0:
                    if data[i-1][j]==0 or data[i+1][j]==0 or data[i][j-1]==0 or data[i][j+1]==0:
                        b.append(i*25+j)
                        xm.append(i)
                        ym.append(j)
        for i in range(len(xm)):
            data[xm[i]][ym[i]]=0             
    b = np.flipud(b)
    b=np.append(b,range(625))
    l1=list(b)
    for el in range(len(l1)-1, -1, -1):
        if l1.count(l1[el]) > 1:
            l1.pop(el)
    return l1

In [3]:
def find_stable_point(md = 'x', mode = 'high', phase = 1):
    res = np.zeros(625)
    res=res-99
    if md=='x':
        path1 ='x/result-'+mode+'-'+str(phase)+'/'
    if md == 'y':
        path1 ='y/result-'+mode+'-'+str(phase)+'/'
    b=[]
    xm=[]
    ym=[]
    filename= './sr_C1_20/800rpm_100kPa_30C_Swirl=20_B00006_PIV_MP(2x32x32_50ov)=unknown/model-3-3-prediction-3-to-6/x/1-raw.dat'
    data=pd.read_csv(filename,index_col = 0).values
    # print(data)
    for k in range(25):
        for i in range(1,24):
            for j in range(1,24):
                #for any sequence in i and j
                if data[i][j]!=0:
                    if data[i-1][j]==0 or data[i+1][j]==0 or data[i][j-1]==0 or data[i][j+1]==0:
                        b.append(i*25+j)

                        xm.append(i)
                        ym.append(j)
        for i in range(len(xm)):
            data[xm[i]][ym[i]]=0
                    
    b = np.flipud(b)
    b=np.append(b,range(625))
    l1=list(b)
    for el in range(len(l1)-1, -1, -1):
        if l1.count(l1[el]) > 1:
            l1.pop(el)
    # print(l1)
    return l1

In [4]:
# print(find_stable_point('x','high',6))

In [5]:

def second_phase_prediction(direction,vec_num,mtype,phase,data1,nxt_phase,scope):
    req_column = len(data1.columns)-1
    path1='./a/'+direction+'/result-'+mtype+'-'+str(phase)+'/'+str(phase-scope+1)+'-'+str(phase)+'-'+str(nxt_phase)+'/tgModels'+str(vec_num)
    isExists=os.path.exists('./a/'+direction+'/result-'+mtype+'-'+str(phase)+'/'+str(phase-scope+1)+'-'+str(phase)+'-'+str(nxt_phase))
    if not isExists:

        os.makedirs('./a/'+direction+'/result-'+mtype+'-'+str(phase)+'/'+str(phase-scope+1)+'-'+str(phase)+'-'+str(nxt_phase))

    save_path = './a/'+direction+'/result-'+mtype+'-'+str(phase)+'/'+str(phase-scope+1)+'-'+str(phase)+'-'+str(nxt_phase)+'/'+str(vec_num)
    isExists2 = os.path.exists(path1)
    if isExists2:
        return
    
        
    data_property=data1[req_column].describe()
    if(data1[req_column].max()==0):
        return
    # train_data.index.duplicated(keep=False)
    data_property.to_csv(save_path+'_data.csv')
    data1.index = range(116*scope*90)
    # 
    predictor_target = TabularPredictor(label=req_column, path=path1).fit(data1, time_limit=180, hyperparameters = { 'CAT': {}, 'XGB': {}, 'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, {}, 'GBMLarge']})
    performance = predictor_target.evaluate(data1)
    result=predictor_target.leaderboard(data1, silent=True)
    result.to_csv(save_path+'.csv')

In [6]:
def get_path(mtype = 'high', sequence = 1):
    if mtype == 'high':
        path1='sr_C1_20'
        swirl = '20'
    elif mtype == 'medium':
        path1= 'sr_C4_12'
        swirl = '12'
    elif mtype == 'low':
        path1 = 'sr_C6_01'
        swirl = '01'
    path2="800rpm_100kPa_30C_Swirl="+swirl+"_B"+str(sequence).zfill(5)+"_PIV_MP(2x32x32_50ov)=unknown/"
    path = path1 + '/' + path2
    return path

In [7]:
def phasewise_prediction(md = 'x',model = 'high',phase = 1,next_phase = 2,scope =1):

    stable_priority = find_stable_point(md, model, next_phase)
    pre_data = read_data(md,model,phase,scope)

    
    pre_data.index = range(scope*90*116)
    
    # pre_data.to_csv('pre_data.csv')    
    nxt_data = read_data(md,model,next_phase,scope)
    nxt_data.index = range(scope*90*116)

    for i in range(625):

        pre_data.insert(len(pre_data.columns),i+625,nxt_data[stable_priority[i]])

        second_phase_prediction(md,i,model,phase,pre_data,next_phase,scope)
        
    #priority list_of stable_point

In [8]:
def crank_angle_trainer(md,begin_CAD,end_CAD,next_CAD):
    phase = 151+int(end_CAD/2)
    scope = int((end_CAD-begin_CAD)/2)+1
    next_phase = 151+int(next_CAD/2)
    
    print(md,phase,next_phase,scope)
    phasewise_prediction('x',md,phase,next_phase,scope)
    phasewise_prediction('y',md,phase,next_phase,scope)

In [9]:
def get_path(mtype = 'high', sequence = 1):
    if mtype == 'high':
        path1='sr_C1_20'
        swirl = '20'
    elif mtype == 'medium':
        path1= 'sr_C4_12'
        swirl = '12'
    elif mtype == 'low':
        path1 = 'sr_C6_01'
        swirl = '01'
    path2="800rpm_100kPa_30C_Swirl="+swirl+"_B"+str(int(sequence)).zfill(5)+"_PIV_MP(2x32x32_50ov)=unknown/"
    path = path1 + '/' + path2
    return path


def read_data(model = 'x',mtype = 'high',phase = 1,scope = 1):
    series = pd.DataFrame(None,columns=range(625))
    train_cycles = []
    print(phase)
    for i in range(10):
        for j in range(1,10):
            train_cycles.append(10*i+j)
    for j in range(116):
        for i in train_cycles:            
            new_series = read_batch(get_path(mtype, phase + j) + 'B' + str(i).zfill(5), model)
            series = pd.concat([series, new_series], ignore_index=True)

    
    series.index = range(116*90*scope)
    return series


def read_batch(str1 = 'B00001',model = 'x'):
    suf_1 = '.dat'
    

    data1 = pd.read_csv(str1+suf_1,skiprows = 3,header = None,delimiter = ' ')
    
    value = data1.values.reshape(25,25,-1)
    if model == 'x':
        vx=pd.DataFrame(value[:,:,2].reshape(-1)).transpose()
    if model == 'y':
        vx=pd.DataFrame(value[:,:,3].reshape(-1)).transpose()
#     y=value[:,:,1]
#     vx=value[:,:,2]
#     vy = value[:,:,3]
    return vx


def single_phase_predictor(mtype = 'low',phase = 1):
    data1 = read_data('x',mtype, phase)
    data2 = read_data('y',mtype, phase)
    # print(data1)
    for target_column in range(625):
        in_phase_prediction('x',target_column,mtype,phase,data1)
        in_phase_prediction('y',target_column,mtype,phase,data2)




In [10]:
# single_phase_predictor('high',23)

In [11]:
def batch_prediction(mtype = 'high'):
    data = pd.read_csv("lab.csv")
    begin_CAD = data['begin_CAD']
    end_CAD = data['end_CAD']
    next_CAD = data['next_CAD']
    predict_CAD = data['predict_CAD']
    for i in range(data.shape[0]):
        crank_angle_trainer(mtype,begin_CAD[i],end_CAD[i],next_CAD[i])

In [12]:
crank_angle_trainer('high',-300,-300,-290)

high 1 6 1
1
6


Beginning AutoGluon training ... Time limit = 180s
AutoGluon will save models to "./a/x/result-high-1/1-1-6/tgModels0\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   71.51 GB / 2000.36 GB (3.6%)
Train Data Rows:    10440
Train Data Columns: 625
Label Column: 625
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (12.5053016814, -14.0150741749, -0.9542, 2.68203)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    62431.04 MB
	Train Data (Original)  Memo

	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Useless Original Features (Count: 315): ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '170', '171', '172', '173', '174', '175', '176', '177', '178', '196', '197', '198', '199', '200', '2

		These features carry no predictive signal and should be manually investigated.
		This is typically a feature which has the same value for all rows.
		These features do not need to be present at inference time.
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 312 | ['83', '84', '85', '86', '87', ...]
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 312 | ['83', '84', '85', '86', '87', ...]
	0.6s = Fit runtime
	312 features in original data used to generate 312 features in processed data.
	Train Data (Processed) Memory Usage: 26.06 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.7s ...
AutoGluon will gauge predictive performance using evaluation metric: 'root_mean_squared_error'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.
	To change this, specify the eval_metric parameter of Predictor()


	To change this, specify the eval_metric parameter of Predictor()
Automatically generating train/validation split with holdout_frac=0.1, Train Rows: 9396, Val Rows: 1044
User-specified model hyperparameters to be fit:
{
	'CAT': {},
	'XGB': {},
	'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, {}, 'GBMLarge'],
}
Fitting 5 L1 models ...
Fitting model: LightGBMXT ... Training model for up to 178.86s of the 178.86s of remaining time.


...

[1000]	valid_set's rmse: 0.756646


	-0.7562	 = Validation score   (-root_mean_squared_error)
	6.56s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 162.59s of the 162.58s of remaining time.
	-0.7391	 = Validation score   (-root_mean_squared_error)
	95.73s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: XGBoost ... Training model for up to 66.71s of the 66.71s of remaining time.
	-0.7995	 = Validation score   (-root_mean_squared_error)
	43.47s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: LightGBMLarge ... Training model for up to 22.78s of the 22.78s of remaining time.
	Ran out of time, early stopping on iteration 745. Best iteration is:
	[739]	valid_set's rmse: 0.806185
	-0.8062	 = Validation score   (-root_mean_squared_error)
	23.02s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 179.25s of the -0.8s of remaining time.
	-0.7308	 = Validation score   (-root_me

AutoGluon will save models to "./a/y/result-high-1/1-1-6/tgModels391\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   44.00 GB / 2000.36 GB (2.2%)
Train Data Rows:    10440
Train Data Columns: 1016
Label Column: 1016
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (33.1785590645, -27.8655930673, 1.0742, 4.21199)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    60651.53 MB
	Train Data (Original)  Memory Usage: 84.86 MB (0.1% of available memory)
	I

AutoGluon will save models to "./a/y/result-high-1/1-1-6/tgModels541\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   43.94 GB / 2000.36 GB (2.2%)
Train Data Rows:    10440
Train Data Columns: 1166
Label Column: 1166
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (11.9937730205, -8.2871147992, -0.64812, 1.6166)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    60617.59 MB
	Train Data (Original)  Memory Usage: 97.38 MB (0.2% of available memory)
	I

	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -0.5698339793965238,
    "mean_squared_error": -0.3247107640748759,
    "mean_absolute_error": -0.4034898376991716,
    "r2": 0.8757391860132798,
    "pearsonr": 0.9405444013631103,
    "median_absolute_error": -0.28943365138623656
}
C:\Users\ps\AppData\Local\Temp\ipykernel_4548\1564152145.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pre_data.insert(len(pre_data.columns),i+625,nxt_data[stable_priority[i]])
C:\Users\ps\AppData\Local\Temp\ipykernel_4548\1564152145.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times